### Biblioteki

In [1]:
from collections import Counter

from datetime import datetime
import datetime as dt

import glob

import math
import numpy as np
import os
import pandas as pd

# Do otwarcia zip z url

import requests
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

### Pliki

`Plik główny`

In [3]:
## PLIK Z LIKWIDACJAMI
path = './likwidacje/'
likwidacje= pd.concat([pd.read_csv(path+part, sep=";", decimal=",") for part in os.listdir(path)])

# NAGŁÓWKI
likwidacje = likwidacje.rename(columns={'Miesiąc utworzenia':'CALMONTH','Sklep':'STORE','Artykul':'ARTICLE','Dzień utworzenia':'CALDAY','Informacja inw.':'TYPE','Wartość':'VALUE'}).reset_index().drop('index',axis=1)

# TYP DANYCH
likwidacje['CALDAY'] = pd.to_datetime(likwidacje['CALDAY'], format='%Y%m%d')

`Plik z kategoriami poduktowymi`

In [83]:
## PLIK Z KATEGORIAMI 
kategorie = pd.read_csv('kategorie.csv', encoding = 'utf-8', sep=';', decimal=',', dtype ={'Artykul':int}).dropna(axis = 0, how = 'all')

`Czyszczenie pliku głównego i łączenie z kategoriami`

In [84]:
# Łączenie z kategoriami
likwidacje = likwidacje.merge(kategorie[['ARTICLE','ARTICLE_CODE']].drop_duplicates(), on='ARTICLE_CODE', how='left')

# WYWALANIE OUTLIERÓW, KTÓRE SĄ NIERACJONALNYMI WARTOŚCIAMI CO DO WARTOŚCI BEZWGLĘDNEJ
likwidacje = likwidacje[~(likwidacje.VALUE.abs() >1000000)]

`Czyszczenie likwidacji z outlierów`

* likwidacje_pozostale - przechowuje informacje o stratach podlegających pod ubezpieczenie 
* likwidacje_inwentaryzacje - przechowuje informacje o stratach z tytułu inwentaryzacji 
* likwidacje - nie zawierają strat podlegających pod ubezpieczenie  i z tytułu inwentaryzacji

W przypadku chęci stworzenia pliku _DANE_STARTOWE_INWENTARYZACJE.csv_ należy postawić symbol __#__ przed resztą linijek i odpowiednio wstawić _likwidacje_inwentaryzacje_ w zaznaczonym niżej miejscu

In [85]:
likwidacje_pozostale = ...
likwidacje_inwentaryzacje = ...
likwidacje = ...


`Czyszczenie likwidacji z odstającymi rodzajów dokumentów`

* likwidacje_odstajace - przechowuje informacje o stratach posiadających bardzo nieregularny wzorzec 
* likwidacje - nie zawierają strat posiadających bardzo nieregularny wzorzec

W przypadku chęci stworzenia pliku _DANE_STARTOWE_ODSTAJACE.csv_ należy postawić symbol __#__ przed resztą linijek i odpowiednio wstawić _likwidacje_odstajace_ w zaznaczonym niżej miejscu

<div class="alert-success">
TUTAJ ODPOWIEDNIO ZMIENIĆ W ZALEŻNOŚCI OD POTRZEB
</div>

Należy uzupełnić według wzoru:  _likiwdacje_ = _ _ _ _ _ _ _, gdzie należy wstawić:  
* likwidacje - w celu wygenerowania _'DANE_STARTOWE_OCZYSZCZONE.csv'_
* likwidacje_inwentaryzacje - w celu wygenerowania _'DANE_STARTOWE_INWENTARYZACJE.csv'_
* likwidacje_odstajace - w celu wygenerowania _'DANE_STARTOWE_ODSTAJACE.csv'_

In [87]:
# Grupowanie bez podziału na kategorie
likwidacje = likwidacje.groupby(['STORE','CALMONTH','CALDAY'])[['STORE','VALUE', 'TYPE', 'CALDAY', 'CALMONTH']].agg({'VALUE':'sum', 'TYPE': (lambda x: Counter(x))}).reset_index()

### Dane zewnętrzne

`Charakterystyki dni`

In [89]:
kalendarz = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'zmienne_swieta')

# Do ustawienia
kalendarz = kalendarz[(kalendarz.data>'2019-06-30')&(kalendarz.data<'2023-02-01')]

# Połączenie
likw = []
for sklep in likwidacje.STORE.unique():
    dla_sklepu = likwidacje[likwidacje.STORE == sklep]
    tab = pd.merge(kalendarz,dla_sklepu, left_on='data', right_on='CALDAY', how='left')
    tab['STORE'] = sklep
    tab['CALDAY'] = tab['data']
    tab['CALMONTH'] = (tab['CALDAY'].dt.year.astype(str) + tab['CALDAY'].dt.month.map("{:02}".format).astype(str)).astype(int)
    likw.append(tab)
likwidacje = pd.concat(likw)

`Plik z informacjami o sklepach własnych`

In [91]:
sklepy_informacje = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'dane_sklepy_dc')

# Dodanie informacji o sklepach do likwidacji
likwidacje = likwidacje.merge(sklepy_informacje, on = 'STORE', how = 'left')
likwidacje = likwidacje[likwidacje.STORE.isin(list(sklepy_informacje.STORE.unique()))]

`Plik z numerami sklepów SAPowymi`

In [92]:
numery = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'numeracja_sklepy_dc', header = 1)
numery = numery.rename(columns = {'Sklep (Kod)': 'STORE','Numer sklepu Detal':'NUMER_SKLEPU'})
numery = numery.dropna()

`Plik z datami inwentaryzacji`

In [93]:
inwentaryzacje = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'daty_inwentaryzacji_dc')
inwentaryzacje['INVENTORY'] = 1

`Dane pogodowe`

In [94]:
stacje = [100, 105, 115, 120, 125, 135, 155, 160, 185, 195, 200, 205, 210, 230, 235, 250, 270, 272, 280, 295, 300, 310, 330, 360, 375, 385, 399, 400, 415, 418, 424, 435, 455, 465, 469, 488, 495, 497, 500, 510, 520, 530, 540, 550, 560, 566, 570, 575, 580, 585, 595, 600, 625, 628, 650, 660, 670, 690]
miesiące = ['01','02','03','04','05','06','07','08','09','10','11','12']
start = 2019
lata = []
for i in range(4):
    lata.append(start+i)
    
# DO 2022 ROKU WŁĄCZNIE

appended_data = []
kolumny_pogoda = pd.read_excel('dane_predykcje.xlsx', sheet_name= 'pogoda_kolumny', header = None)
kolumny_do_usunięcia = ['Status pomiaru NOS', 'Status pomiaru FWS', 'Status pomiaru TEMP', 'Status pomiaru CPW', 'Status pomiaru WLGS','Status pomiaru PPPS', 'Status pomiaru PPPM', 'Status pomiaru WODZ', 'Status pomiaru WONO']
for i in stacje:
    for ii in lata:
        try:
            r = requests.get(f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/{str(ii)}/{str(ii)}_{str(i)}_s.zip")
            files = ZipFile(BytesIO(r.content))
            pogoda = pd.read_csv(files.open(f"s_d_t_{str(i)}_{str(ii)}.csv"), header = None, encoding = 'windows-1250')
            pogoda.columns = list(kolumny_pogoda.iloc[:,0])
            pogoda.drop(kolumny_do_usunięcia, axis=1, inplace=True)
            appended_data.append(pogoda)
        except:
            continue

appended_data = pd.concat(appended_data)

# ROK 2023 - W RAZIE JAKBY ZMIENIŁO SIĘ Z NOWYM ROKIEM TO TO ZMIENIĆ!

appended_data2 = []
for i in miesiące:
        try:
            r = requests.get(f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/synop/2023/2023_{i}_s.zip")
            files = ZipFile(BytesIO(r.content))
            pogoda = pd.read_csv(files.open(f"s_d_t_{i}_2023.csv"), header = None, encoding = 'windows-1250')
            pogoda.columns = list(kolumny_pogoda.iloc[:,0])
            pogoda.drop(kolumny_do_usunięcia, axis=1, inplace=True)
            appended_data2.append(pogoda)
        except:
            continue

appended_data2 = pd.concat(appended_data2)

# ZŁĄCZONE DANE ZA WSZYSTKIE LATA

pogoda = pd.concat([appended_data, appended_data2], ignore_index=True)

# Errata do danych
'''
Stacja Katowice-Muchowiec do 31.12.2018 to stacja Katowice
Stacja Łóź-Lublinek do 31.12.2018 to stacja Łóź
Stacja Poznań-Ławica do 31.12.2018 to stacja Poznań
Stacja Warszawa-Okęcie do 31.12.2018 to stacja Warszawa
Stacja Wrocław-Strachowice do 31.12.2018 to stacja Wrocław
Stacja Elbląg-Milejewo do 31.03.2013 to stacja Elbląg w inej lokalizacji, system nie pozwala przechowywać podwójnej nazwy stacji dla tego samego kodu.
Stacja Resko-Smólsko do 31.12.2014 to stacja Resko.
Stacja Kołobrzeg-Dźwirzyno do 8.04.2018 to stacja Kołobrzeg, nastąpiła zmiana lokalizacji stacji z zachowaniem kodu.
''' ;    

pogoda['Nazwa stacji'] = pogoda['Nazwa stacji'].apply(lambda x: 'KOŁOBRZEG' if x == 'KOŁOBRZEG-DŹWIRZYNO' else 
                                                      ('WARSZAWA' if x == 'WARSZAWA-OKĘCIE' else
                                                      ('RESKO' if x == 'RESKO-SMÓLSKO' else
                                                      ('ELBLĄG' if x == 'ELBLĄG-MILEJEWO' else
                                                      ('POZNAŃ' if x == 'POZNAŃ-ŁAWICA' else
                                                      ('ŁÓDŹ' if x == 'ŁÓDŹ-LUBLINEK' else
                                                      ('KATOWICE' if x == 'KATOWICE-MUCHOWIEC' else
                                                      ('WROCŁAW' if x == 'WROCŁAW-STRACHOWICE' else x))))))))                                              
                                                

# Obróbka kolumn przed dołączeniem historycznych informacji o pogodzie
pogoda['CALDAY'] = pd.to_datetime(dict(year=pogoda.Rok, month=pogoda.Miesiąc, day=pogoda.Dzień))
pogoda = pogoda.rename(columns = {'Nazwa stacji':'STACJA_POGODOWA'})

# Kolumny do połączenia
pogoda_kolumny = ['Kod stacji', 'STACJA_POGODOWA', 'Średnie dobowe zachmurzenie ogólne [oktanty]', 'Średnia dobowa prędkość wiatru [m/s]',
 'Średnia dobowa temperatura [°C]', 'Średnia dobowa wilgotność względna [%]', 'Średnia dobowe ciśnienie na poziomie stacji [hPa]', 
 'Średnie dobowe ciśnienie na pozimie morza [hPa]', 'Suma opadu dzień  [mm]', 'Suma opadu noc   [mm]', 'CALDAY']

# Połączenie
likwidacje = likwidacje.merge(pogoda[pogoda_kolumny], on = ['CALDAY','STACJA_POGODOWA'], how='left')
likwidacje = likwidacje.dropna(subset='STACJA_POGODOWA')

`Inflacja`

In [95]:
inflacja = pd.read_csv("https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultstronaopisowa/4741/1/1/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.csv", encoding = 'windows-1250', sep=';', decimal=',')

# USUNIĘCIE BEZSENSOWNYCH KOLUMN
inflacja = inflacja.iloc[:,:6]

# STWORZENIE KOLUMNY Z PRZYJAZNYM FORMATEM I NOWEJ Z OPÓŹNIENIEM ZMIENNEJ
inflacja['Wskaźnik_inflacji'] = inflacja['Wartość'] - 100
inflacja['Wskaźnik_inflacji_lag'] = inflacja['Wskaźnik_inflacji'].shift(1)

# Obróbka ramki i kolumn
inflacja = inflacja[(inflacja['Sposób prezentacji'] == 'Analogiczny miesiąc poprzedniego roku = 100') & (inflacja['Rok']>2015)]
inflacja['Miesiąc'] = inflacja.Miesiąc.map("{:02}".format)
inflacja['CALMONTH'] = inflacja.Rok.astype(str) + inflacja['Miesiąc']
inflacja['CALMONTH'] = inflacja['CALMONTH'].astype(int)

# Połączenie
likwidacje = likwidacje.merge(inflacja[['Wartość', 'Wskaźnik_inflacji', 'Wskaźnik_inflacji_lag', 'CALMONTH']], on = 'CALMONTH', how='left').rename(columns = {'Wartość':'Wartość inflacji'})

`Bezrobocie|`

In [96]:
# Źródło: https://stat.gov.pl/obszary-tematyczne/rynek-pracy/bezrobocie-rejestrowane/bezrobotni-oraz-stopa-bezrobocia-wg-wojewodztw-podregionow-i-powiatow---styczen-grudzien-2004-r,2,3.html?contrast=default

In [3]:
bezrobocie = pd.read_excel('dane_predykcje.xlsx', sheet_name='bezrobocie_woj')

bezrobocie = bezrobocie.drop(['WOJ.','POW.'], axis =1).set_index('powiat').stack().reset_index().rename(columns={'powiat':'WOJEWÓDZTWO','level_1':'CALMONTH',0:'UNEMP_RATE'})
bezrobocie = bezrobocie.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Połączenie 
likwidacje  = likwidacje.merge(bezrobocie, on = ['WOJEWÓDZTWO','CALMONTH'], how = 'left')

`Pozostałe dane makroekonomiczne publikowane dla kwartałów`

In [98]:
makroekonomiczne = pd.read_excel('dane_predykcje.xlsx', sheet_name='dane_makro').set_index('index').stack().reset_index().rename(columns={'level_1':'kwartał',0:'wartość'})

# Ogranieczenie kwartałów i usunięcie niepotrzebnych spacji w wartościach
makroekonomiczne = makroekonomiczne[~makroekonomiczne.kwartał.isin(['2022','2023','2024','2025'])].apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Zamiana tekstu na datę - kwartał
makroekonomiczne['kwartał'] =  pd.to_datetime(makroekonomiczne['kwartał']).dt.to_period('Q')
# Stworzenie ostatecznej tabelki 
makroekonomiczne = makroekonomiczne.pivot_table('wartość', 'kwartał','index').reset_index()

# stworzenie w danych kolumny z kwartałem
likwidacje['kwartał'] = pd.PeriodIndex(likwidacje.CALDAY, freq='Q')

# połączenie
likwidacje = likwidacje.merge(makroekonomiczne, on='kwartał', how = 'left')

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Dane wewnętrzne

`Stany magazynowe`

In [99]:
path = './zapasy/' # zapasy siedzą w folderze zapasy
zapasy = pd.concat([pd.read_csv(path+part, sep="\t", decimal=",", encoding='cp1250') for part in os.listdir(path)])

# Połączenie
likwidacje = likwidacje.merge(zapasy[['CALDAY', 'STORE', 'STOCK_VALUE',
       'STOCK_QUANTITY']], on = ['CALDAY','STORE'], how='left')

`Nałożenie inwentaryzacji`

In [100]:
# Połączenie - nałożenie inwentaryzacji
likwidacje = likwidacje.merge(inwentaryzacje[['STORE','INVENTORY_DATE','INVENTORY']].drop_duplicates(), left_on = ['STORE','CALDAY'], right_on = ['STORE','INVENTORY_DATE'], how='left').drop('INVENTORY_DATE', axis=1)
likwidacje['INVENTORY'].fillna(0, inplace= True)

`Sprzedaż paragonowa`

In [101]:
path = './sprzedaz/' # paragony siedzą w folderze sprzedaz
paragony= pd.concat([pd.read_parquet(path+part) for part in os.listdir(path)])

# KOLUMNA Z MIESIĄCEM - czyszczenie kolumn
paragony['CALMONTH'] = paragony['CALDAY'].apply(lambda x: str(x)[:-2])
paragony['ARTICLE'] = pd.to_numeric(paragony.ARTICLE)
paragony['CALDAY'] = pd.to_datetime(paragony['CALDAY'])

# Dodanie kategorii
paragony = paragony.merge(kategorie[['ARTICLE_CODE','ARTICLE']].drop_duplicates(), on='ARTICLE_CODE', how='left')

# Grupowanie
paragony = paragony.groupby(['STORE','CALMONTH','CALDAY'])[['STORE','TOTAL_SALES', 'TOTAL_VOLUME_SOLD', 'CALDAY', 'CALMONTH']].agg({'TOTAL_SALES':'sum','TOTAL_VOLUME_SOLD': 'sum'}).reset_index()
paragony['STORE'] = paragony['STORE'].astype(int)

# Ewentualnie do korekty
paragony.CALMONTH = paragony.CALMONTH.astype(int)

# Połączenie
likwidacje = pd.merge(likwidacje, paragony, on=['STORE','CALMONTH', 'CALDAY'], how='left')

`Nałożenie zmiany numerów sklepów w czasie`

`Zapisanie danych do pliku`

In [103]:
likwidacje.to_csv('DANE_STARTOWE_OCZYSZCZONE.csv', index = False)
likwidacje.to_csv('DANE_STARTOWE_INWENTARYZACJE.csv', index = False)
likwidacje.to_csv('DANE_STARTOWE_ODSTAJACE.csv', index = False)
likwidacje.to_csv('DANE_STARTOWE_WSZYSTKIE.csv', index = False)